<img src=http://fd.perso.eisti.fr/Logos/TORUS2.png>




# Exercise time!

So, after all the lessons, it's your turn now to practice how to build a linear regression model in Spark!

### Exercise 1 : 

In HDFS of factory02 : "hdfs://hupi-factory-02-01-01-01/user/hupi/dataset_torusVN/geoVN.json", we have a small dataset (40 observations) of some characteristics of climate of some cities in Vietnam (source : https://www.meteoblue.com/en/weather/). 

Let's try building a linear regression model that predicts the temperature daily mean thanks to 8 variables below : 
- High cloud cover daily mean
- Low cloud cover daily mean
- Mean Sea Level Pressure daily mean
- Medium cloud cover daily mean
- Relative humidity daily mean
- Shortwave Radiation - backwards daily sum
- Total Precipitation daily sum
- Total cloud cover daily mean

To put it simply, we'll just care about these numerical variables; otherwise in this dataset, we can also inspect the relation between temperature daily mean and the city (categorical variable).

### Exercise 2 : 

Create an API that can predict and give the result in the interface of Hupi. (suggestion : convert model to PMML format -> create an API in Hupi - Predict -> create a Widget...)

### Exercise 3 :

With the dataset geoVN.json, let's create some visualizations in Hupi !

In [ ]:
// First initialize the system

import org.apache.spark.sql.SparkSession

val sparkSession = SparkSession
  .builder()
  .appName("linearRegModel_florent")
  .getOrCreate()

// For implicit conversions like converting RDDs to DataFrames
import sparkSession.implicits._

import org.apache.spark.sql.SparkSession
sparkSession: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5f8775d8
import sparkSession.implicits._


In [ ]:
// Read the file
val df = sparkSession.read.json(sparkContext.wholeTextFiles("hdfs://hupi-factory-02-01-01-01/user/hupi/dataset_torusVN/geoVN.json").values)

df: org.apache.spark.sql.DataFrame = [City: string, Day: bigint ... 15 more fields]


In [ ]:
// Statistics
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.stat.{MultivariateStatisticalSummary, Statistics}

import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.stat.{MultivariateStatisticalSummary, Statistics}


In [ ]:
// Convert df to RDD to be able to use the library MultiVariateStatisticalSummary. Here we choose only the necessary variables
val rdd = df.drop("City").drop("Day").drop("Year").drop("Month").drop("timestamp").drop("Lat").drop("Lon").drop("datestamp")
            .map(l => (l(6).asInstanceOf[Double], l(1).asInstanceOf[Double], l(2).asInstanceOf[Double],
                       l(3).asInstanceOf[Double], l(4).asInstanceOf[Double], l(5).asInstanceOf[Double],
                       l(0).asInstanceOf[Double], l(7).asInstanceOf[Double], l(8).asInstanceOf[Double])).rdd


rdd: org.apache.spark.rdd.RDD[(Double, Double, Double, Double, Double, Double, Double, Double, Double)] = MapPartitionsRDD[9] at rdd at <console>:87


In [ ]:
// Convert rdd to the rdd of vectors
val observations = rdd.map(l => Vectors.dense(l._1, l._2, l._3, l._4, l._5, l._6, l._7, l._8, l._9))

observations: org.apache.spark.rdd.RDD[org.apache.spark.mllib.linalg.Vector] = MapPartitionsRDD[10] at map at <console>:86


In [ ]:
// Compute column summary statistics.
val summary: MultivariateStatisticalSummary = Statistics.colStats(observations)


summary: org.apache.spark.mllib.stat.MultivariateStatisticalSummary = org.apache.spark.mllib.stat.MultivariateOnlineSummarizer@27969cb7


In [ ]:
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.stat.Statistics
import org.apache.spark.rdd.RDD

import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.stat.Statistics
import org.apache.spark.rdd.RDD


In [ ]:
// calculate the correlation matrix using Pearson's method. Use "spearman" for Spearman's method
// If a method is not specified, Pearson's method will be used by default.
val correlMatrix: Matrix = Statistics.corr(observations, "pearson")


correlMatrix: org.apache.spark.mllib.linalg.Matrix =
1.0                   -0.6071271749991745  -0.7769139178723304    ... (9 total)
-0.6071271749991745   1.0                  0.15213518644495438    ...
-0.7769139178723304   0.15213518644495438  1.0                    ...
-0.6317688824653139   0.5795801157147833   0.3987737633967863     ...
-0.8783589380133455   0.7848585544175861   0.5355160905617118     ...
0.7788360980837538    -0.7321669112163407  -0.5401236564390007    ...
-0.07984925706043007  0.34275011990685916  -0.054919658480978115  ...
-0.3896111580334033   0.6302167222500573   0.17145792931429643    ...
-0.6190037706669987   0.6470804476922558   0.3584018763482733     ...
